In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [2]:
# Get the current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)

# Define the relative paths to your directories without duplication
train_dir = os.path.join(current_directory, 'training')
validation_dir = os.path.join(current_directory, 'validation')
test_dir = os.path.join(current_directory, 'testing')

# Check the full paths
print("Full path to training directory:", train_dir)
print("Full path to validation directory:", validation_dir)
print("Full path to testing directory:", test_dir)


Current working directory: c:\Users\devan\OneDrive\Desktop\MP\Proccessed dataset
Full path to training directory: c:\Users\devan\OneDrive\Desktop\MP\Proccessed dataset\training
Full path to validation directory: c:\Users\devan\OneDrive\Desktop\MP\Proccessed dataset\validation
Full path to testing directory: c:\Users\devan\OneDrive\Desktop\MP\Proccessed dataset\testing


In [4]:
# Set image dimensions (adjust if necessary)
img_width, img_height = 224, 224  # Common image size for CNNs

# Data augmentation for improving generalization (optional)
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # Adjust batch size based on GPU memory
    class_mode='binary'  # Binary classification (real vs. fake)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)


Found 121159 images belonging to 2 classes.
Found 53184 images belonging to 2 classes.


In [5]:
# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout for regularization

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  

model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification


In [6]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
# Train the model
model.fit(
    train_generator,
    epochs=10,  # Adjust epochs based on validation performance
    validation_data=validation_generator
)

Epoch 1/10


3787/3787 [==============================] - 2389s 631ms/step - loss: 0.5996 - accuracy: 0.6746 - val_loss: 0.5667 - val_accuracy: 0.6921
Epoch 2/10
3787/3787 [==============================] - 2355s 622ms/step - loss: 0.5464 - accuracy: 0.7152 - val_loss: 0.5512 - val_accuracy: 0.7209
Epoch 3/10
3787/3787 [==============================] - 2525s 667ms/step - loss: 0.5272 - accuracy: 0.7300 - val_loss: 0.5533 - val_accuracy: 0.7184
Epoch 4/10
3787/3787 [==============================] - 2651s 700ms/step - loss: 0.5134 - accuracy: 0.7411 - val_loss: 0.5393 - val_accuracy: 0.7323
Epoch 5/10
3787/3787 [==============================] - 2557s 675ms/step - loss: 0.5017 - accuracy: 0.7503 - val_loss: 0.4871 - val_accuracy: 0.7660
Epoch 6/10
3787/3787 [==============================] - 2469s 652ms/step - loss: 0.4923 - accuracy: 0.7574 - val_loss: 0.4700 - val_accuracy: 0.7814
Epoch 7/10
3787/3787 [==============================] - 2441s 645ms/step - loss: 0.4852 - accuracy: 0.76

In [8]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Found 35304 images belonging to 2 classes.
1104/1104 [==============================] - 111s 101ms/step - loss: 0.4831 - accuracy: 0.7683
Test accuracy: 0.7682982087135315
